In [45]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [46]:
ori=pd.read_csv('website_data_20190225.csv')
ori.drop(['STATE','DISTRICT','WLCODE','SITE_TYPE','TEH_NAME'],axis=1,inplace=True)
ori.replace(to_replace="'0",value=0,inplace=True)
# ori.head()

In [47]:
dataset=pd.DataFrame().reindex_like(ori)
dataset1=pd.DataFrame().reindex_like(ori)
dataset.dropna(inplace=True)
dataset1.dropna(inplace=True)

# j=0
# for i in range(0,ori.shape[0]):
#     if ori['STATE'][i]=='RJ':
#         dataset.loc[j] = ori.iloc[i]
#         j+=1
# dataset.drop(['STATE'],axis=1,inplace=True)

# j=0
# for i in range(0,ori.shape[0]):
#     if ori['DISTRICT'][i]=='Ajmer':
#         dataset.loc[j] = ori.iloc[i]
#         j+=1
# dataset.drop(['DISTRICT'],axis=1,inplace=True)

j=0
for i in range(0,ori.shape[0]):
    if ori['BLOCK_NAME'][i]=='Arain':
        dataset1.loc[j] = ori.iloc[i]
        j+=1
dataset1.drop(['BLOCK_NAME'],axis=1,inplace=True)
dataset.drop(['BLOCK_NAME'],axis=1,inplace=True)

j=0
for i in range(0,dataset1.shape[0]):
    if dataset1['SITE_NAME'][i]=='Sanpla':
        dataset.loc[j] = dataset1.iloc[i]
        j+=1
lat=dataset["LAT"][0]
lon=dataset["LON"][0]
dataset.drop(['SITE_NAME','LAT','LON'],axis=1,inplace=True)
 
# dataset

In [48]:
for i in range(0,dataset.shape[0]):
    dataset['MONSOON'][i]=float(dataset['MONSOON'][i])
    dataset['POMRB'][i]=float(dataset['POMRB'][i])
    dataset['POMKH'][i]=float(dataset['POMKH'][i])
    dataset['PREMON'][i]=float(dataset['PREMON'][i])
    dataset['YEAR_OBS'][i]=int(dataset['YEAR_OBS'][i])

In [49]:
first=list(dataset['MONSOON'])
second=list(dataset['POMRB'])
third=list(dataset['POMKH'])
fourth=list(dataset['PREMON'])
dataset['MONSOON']=pd.core.frame.DataFrame(x+y+z+w for x, y,z,w in zip(first, second, third, fourth))
dataset.drop(['POMRB','POMKH','PREMON'],axis=1,inplace=True)
dataset = dataset.iloc[::-1]
# dataset

In [50]:
dataset['YEAR_OBS']=(dataset['YEAR_OBS']).apply(np.int64)

In [51]:
dataset['YEAR_OBS']=pd.to_datetime(dataset['YEAR_OBS'],yearfirst=True,format='%Y',infer_datetime_format=True)
indexedDataset=dataset.set_index(['YEAR_OBS'])

In [52]:
from datetime import datetime
# indexedDataset.head(50)

In [53]:
# plt.xlabel('Years')
# plt.ylabel('Water-Level')
# plt.plot(indexedDataset)

- A stationary time series is one whose statistical properties such as mean, variance, autocorrelation, etc. are all constant over time. Most statistical forecasting methods are based on the assumption that the time series can be rendered approximately stationary (i.e., "stationarized") through the use of mathematical transformations. A stationarized series is relatively easy to predict: you simply predict that its statistical properties will be the same in the future as they have been in the past!

- We can check stationarity using the following:

- - Plotting Rolling Statistics: We can plot the moving average or moving variance and see if it varies with time. This is more of a visual technique.
- - Dickey-Fuller Test: This is one of the statistical tests for checking stationarity. Here the null hypothesis is that the TimeSeries is non-stationary. The test results comprise of a Test Statistic and some Critical Values for difference confidence levels. If the ‘Test Statistic’ is less than the ‘Critical Value’, we can reject the null hypothesis and say that the series is stationary.

In [54]:
from statsmodels.tsa.stattools import adfuller
def test_stationary(timeseries):
    
    #Determing rolling statistics
    moving_average=timeseries.rolling(window=12).mean()
    standard_deviation=timeseries.rolling(window=12).std()
    
    #Plot rolling statistics:
#     plt.plot(timeseries,color='blue',label="Original")
#     plt.plot(moving_average,color='red',label='Mean')
#     plt.plot(standard_deviation,color='black',label='Standard Deviation')
#     plt.legend(loc='best')  #best for axes
#     plt.title('Rolling Mean & Deviation')
#     plt.show()
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
#     print('Results Of Dickey-Fuller Test')
    tstest=adfuller(timeseries['MONSOON'],autolag='AIC')
    tsoutput=pd.Series(tstest[0:4],index=['Test Statistcs','P-value','#Lags used',"#Obs. used"])
    #Test Statistics should be less than the Critical Value for Stationarity
    #lesser the p-value, greater the stationarity
    # print(list(dftest))
    for key,value in tstest[4].items():
        tsoutput['Critical Value (%s)'%key]=value
#     print((tsoutput))

In [57]:
test_stationary(indexedDataset)

- There are 2 major reasons behind non-stationaruty of a TS:
- - Trend – varying mean over time. For eg, in this case we saw that on average, the number of passengers was growing over time.
- - Seasonality – variations at specific time-frames. eg people might have a tendency to buy cars in a particular month because of pay increment or festivals.

## Indexed Dataset Logscale

In [58]:
indexedDataset_logscale=np.log(indexedDataset)
test_stationary(indexedDataset_logscale)

## Dataset Log Minus Moving Average (dl_ma)

In [59]:
rolmeanlog=indexedDataset_logscale.rolling(window=12).mean()
dl_ma=indexedDataset_logscale-rolmeanlog
# dl_ma.head(12)

In [60]:
dl_ma.dropna(inplace=True)
# dl_ma.head(12)

In [61]:
test_stationary(dl_ma)

## Exponential Decay Weighted Average (edwa)

In [62]:
edwa=indexedDataset_logscale.ewm(halflife=12,min_periods=0,adjust=True).mean()
# plt.plot(indexedDataset_logscale)
# plt.plot(edwa,color='red')

## Dataset Logscale Minus Moving Exponential Decay Average (dlmeda)

In [63]:
dlmeda=indexedDataset_logscale-edwa
test_stationary(dlmeda)

## Eliminating Trend and Seasonality

- Differencing – taking the differece with a particular time lag
- Decomposition – modeling both trend and seasonality and removing them from the model.

# Differencing

## Dataset Log Div Shifting (dlds)

In [64]:
#Before Shifting
# indexedDataset_logscale.head()

In [65]:
#After Shifting
# indexedDataset_logscale.shift().head()

In [66]:
dlds=indexedDataset_logscale-indexedDataset_logscale.shift()
dlds.dropna(inplace=True)
test_stationary(dlds)

# Decomposition

In [67]:
from statsmodels.tsa.seasonal import seasonal_decompose
decompostion= seasonal_decompose(indexedDataset_logscale,freq=10)

trend=decompostion.trend
seasonal=decompostion.seasonal
residual=decompostion.resid

# plt.subplot(411)
# plt.plot(indexedDataset_logscale,label='Original')
# plt.legend(loc='best')

# plt.subplot(412)
# plt.plot(trend,label='Trend')
# plt.legend(loc='best')

# plt.subplot(413)
# plt.plot(seasonal,label='Seasonal')
# plt.legend(loc='best')

# plt.subplot(414)
# plt.plot(residual,label='Residual')
# plt.legend(loc='best')

# plt.tight_layout() #To Show Multiple Grpahs In One Output, Use plt.subplot(abc)

- Here trend, seasonality are separated out from data and we can model the residuals. Lets check stationarity of residuals:

In [68]:
decomposedlogdata=residual
decomposedlogdata.dropna(inplace=True)
# test_stationary(decomposedlogdata)

# Forecasting a Time Series

- ARIMA stands for Auto-Regressive Integrated Moving Averages. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:

- - Number of AR (Auto-Regressive) terms (p): AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).
- - Number of MA (Moving Average) terms (q): MA terms are lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where e(i) is the difference between the moving average at ith instant and actual value.
- - Number of Differences (d): These are the number of nonseasonal differences, i.e. in this case we took the first order difference. So either we can pass that variable and put d=0 or pass the original variable and put d=1. Both will generate same results.


- An importance concern here is how to determine the value of ‘p’ and ‘q’. We use two plots to determine these numbers.

- - Autocorrelation Function (ACF): It is a measure of the correlation between the the TS with a lagged version of itself-. For instance at lag 5, ACF would compare series at time instant ‘t1’…’t2’ with series at instant ‘t1-5’…’t2-5’ (t1-5 and t2 being end points).
- - Partial Autocorrelation Function (PACF): This measures the correlation between the TS with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons. Eg at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

## ACF & PACF Plots

In [69]:
from statsmodels.tsa.stattools import acf,pacf
lag_acf=acf(dlds,nlags=20)
lag_pacf=pacf(dlds,nlags=20,method='ols')

# plt.subplot(121)
# plt.plot(lag_acf)
# plt.axhline(y=0, linestyle='--',color='gray')
# plt.axhline(y=1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
# plt.axhline(y=-1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
# plt.title('AutoCorrelation Function')

# plt.subplot(122)
# plt.plot(lag_pacf)
# plt.axhline(y=0, linestyle='--',color='gray')
# plt.axhline(y=1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
# plt.axhline(y=-1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
# plt.title('PartialAutoCorrelation Function')

# plt.tight_layout()

- In this plot, the two dotted lines on either sides of 0 are the confidence interevals. These can be used to determine the ‘p’ and ‘q’ values as:

- - p – The lag value where the PACF chart crosses the upper confidence interval for the first time. If we notice closely, in this case p=2.
- - q – The lag value where the ACF chart crosses the upper confidence interval for the first time. If we notice closely, in this case q=2.

In [70]:
from statsmodels.tsa.arima_model import ARIMA

model=ARIMA(indexedDataset_logscale,order=(5,1,0))
results_AR=model.fit(disp=-1)
# plt.plot(dlds)
# plt.plot(results_AR.fittedvalues,color='red')
# plt.title('RSS: %.4f'%sum((results_AR.fittedvalues-dlds['MONSOON'])**2))
# print('Plotting AR Model')

In [71]:
model = ARIMA(indexedDataset_logscale, order=(0, 1, 2))  #0,1,2
results_MA = model.fit(disp=-1)  
# plt.plot(dlds)
# plt.plot(results_MA.fittedvalues, color='red')
# plt.title('RSS: %.4f'%sum((results_MA.fittedvalues-dlds['MONSOON'])**2))
# print('Plotting MA Model')

In [75]:
model = ARIMA(indexedDataset_logscale, order=(5, 1, 1))  
results_ARIMA = model.fit(disp=-1)  
# plt.plot(dlds)
# plt.plot(results_ARIMA.fittedvalues, color='red')
# plt.title('RSS: %.4f'%sum((results_ARIMA.fittedvalues-dlds['MONSOON'])**2))
# print('Plotting Combined Model')

# Taking it back to original scale from residual scale

In [ ]:
#storing the predicted results as a separate series
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
# predictions_ARIMA_diff.head()

- Notice that these start from ‘1949-02-01’ and not the first month. Why? This is because we took a lag by 1 and first element doesn’t have anything before it to subtract from. The way to convert the differencing to log scale is to add these differences consecutively to the base number. An easy way to do it is to first determine the cumulative sum at index and then add it to the base number.

In [37]:
#convert to cummuative sum
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
# predictions_ARIMA_diff_cumsum.head()

YEAR_OBS
1995-01-01   -0.062128
1996-01-01   -0.148681
1997-01-01   -0.080785
1998-01-01   -0.293591
1999-01-01   -0.446932
dtype: float64

In [38]:
predictions_ARIMA_log = pd.Series(indexedDataset_logscale['MONSOON'].ix[0], index=indexedDataset_logscale.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
# predictions_ARIMA_log.head()

YEAR_OBS
1994-01-01    3.411478
1995-01-01    3.349350
1996-01-01    3.262797
1997-01-01    3.330693
1998-01-01    3.117886
dtype: float64

- Here the first element is base number itself and from there on the values cumulatively added.

In [40]:
#Last step is to take the exponent and compare with the original series.
predictions_ARIMA = np.exp(predictions_ARIMA_log)
# plt.plot(indexedDataset)
# plt.plot(predictions_ARIMA)
# plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-indexedDataset['MONSOON'])**2)/len(indexedDataset)))

- Finally we have a forecast at the original scale.

In [41]:
# results_ARIMA.plot_predict(1,26)

#start = !st month
#end = 10yrs forcasting = 144+12*10 = 264th month

#Two models corresponds to AR & MA

In [42]:
x=results_ARIMA.forecast(steps=5)
# print(x)
#values in residual equivalent

(array([1.90599229, 1.69799919, 1.94723178, 1.63484492, 1.69498571]), array([0.4217554 , 0.43999561, 0.46503931, 0.4692189 , 0.47239288]), array([[1.07936689, 2.73261768],
       [0.83562365, 2.56037474],
       [1.03577148, 2.85869207],
       [0.71519277, 2.55449707],
       [0.76911268, 2.62085874]]))


In [43]:
# for i in range(0,5):
#     print(x[0][i],end='')
#     print('\t',x[1][i],end='')
#     print('\t',x[2][i])

1.9059922882826494	 0.42175540014923524	 [1.07936689 2.73261768]
1.6979991933521283	 0.4399956081180564	 [0.83562365 2.56037474]
1.9472317783860675	 0.4650393083562316	 [1.03577148 2.85869207]
1.634844920955585	 0.46921890104261377	 [0.71519277 2.55449707]
1.6949857059390567	 0.47239287904149346	 [0.76911268 2.62085874]


In [44]:
np.exp(x[0])

array([6.72607853, 5.46300603, 7.00925752, 5.12866259, 5.44656811])